In [272]:
import torch
import torchinfo
import timm
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from dataset import *

In [273]:
import torchmetrics

In [355]:
task = "multiclass"
top_k = 1
accuracy = torchmetrics.classification.Accuracy(task=task, threshold=0.5, num_classes=10, top_k=top_k).to("cuda")
precision = torchmetrics.Precision(task=task, threshold=0.5, num_classes=10, top_k=top_k).to("cuda")
recall = torchmetrics.Recall(task=task, threshold=0.5, num_classes=10, top_k=top_k).to("cuda")
f1 = torchmetrics.F1Score(task=task, threshold=0.5, num_classes=10, top_k=top_k).to("cuda")
stat = torchmetrics.StatScores(task=task, threshold=0.5, num_classes=10, top_k=top_k).to("cuda")

In [356]:
preds = torch.tensor([[-0.0129, -0.3175, -0.0683,  0.2080,  0.1129,  0.0553,  0.0140,  0.0251,
         -0.1527,  0.0622],
        [ 0.1304,  0.1785,  0.0883,  0.5066, -0.1161, -0.1496, -0.2035,  0.0709,
          0.2420,  0.3981],
        [-0.0213, -0.0850, -0.1686,  0.1521,  0.2414, -0.0275,  0.0066,  0.1848,
         -0.1766,  0.0321],
        [ 0.3734,  0.3885, -0.2002, -0.3148, -0.2604,  0.3571, -0.7075, -0.2140,
          0.2051,  0.7275],
        [-0.1535, -0.1203, -0.0069,  0.2753,  0.3317, -0.3602, -0.1358,  0.1422,
         -0.0302,  0.3005],
        [ 0.2874,  0.1921,  0.2205,  0.3036, -0.1055, -0.1325,  0.0322,  0.4317,
          0.1032, -0.0067],
        [ 0.0414, -0.1034, -0.1293,  0.0489,  0.3273,  0.1023, -0.1438,  0.1175,
         -0.0843,  0.1121],
        [-0.1684, -0.0975, -0.1321,  0.1957,  0.1615,  0.1282, -0.2379,  0.0721,
         -0.1267,  0.1032],
        [ 0.0425, -0.1601, -0.0020,  0.1313,  0.1604,  0.1026,  0.0792,  0.1182,
         -0.0664,  0.0552],
        [-0.0258, -0.2268, -0.0416,  0.0605,  0.1985,  0.0211, -0.2180,  0.1641,
         -0.0347, -0.0319]]).to("cuda")
target = torch.tensor([3, 7, 0, 1, 3, 8, 9, 8, 3, 6]).to("cuda")
print(stat(preds, target))
print(accuracy(preds, target))
print(precision(preds, target))
print(recall(preds, target))
print(f1(preds, target))

tensor([ 1,  9, 81,  9, 10], device='cuda:0')
tensor(0.1000, device='cuda:0')
tensor(0.1000, device='cuda:0')
tensor(0.1000, device='cuda:0')
tensor(0.1000, device='cuda:0')


In [351]:
print(stat.compute())
print(accuracy.compute())
print(precision.compute())
print(recall.compute())
print(f1.compute())

tensor([ 4, 46, 44,  6, 10], device='cuda:0')
tensor(0.4000, device='cuda:0')
tensor(0.0800, device='cuda:0')
tensor(0.4000, device='cuda:0')
tensor(0.1333, device='cuda:0')


In [276]:
root_path = f"C:/Users/conon/Documents/Datasets/cifar10/"
train_path = os.path.join(root_path, "train")
val_path = os.path.join(root_path, "test")

train_labels_path = os.path.join(root_path, "train_labels.csv")
val_labels_path = os.path.join(root_path, "test_labels.csv")

In [138]:
train = datasets.CIFAR10(root=root_path, train=True, download=True)
val = datasets.CIFAR10(root=root_path, train=False, download=True)

for i, img in enumerate(train.data):
     np.save(os.path.join(root_path + "train", str(i)), img)
pd.DataFrame({"label": train.targets}).to_csv(train_labels_path, index=True)
for i, img in enumerate(val.data):
     np.save(os.path.join(root_path + "test", str(i)), img)
pd.DataFrame({"label": val.targets}).to_csv(val_labels_path, index=True)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
transforms.ToTensor()

In [33]:
import torch.nn.functional as F
F.one_hot(torch.tensor([4]), num_classes=10)

tensor([[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [20]:
pd.read_csv(val_labels_path).nunique()["label"]

10

In [102]:
def weights_ema(model_ema, model, decay=0.999):
    """
    For visualizing and evaliating generator output at any given point during the training
    use an exponential running average for the weights of the generator with decay 0.999.
    """
    par1 = dict(model_ema.named_parameters())
    par2 = dict(model.named_parameters())

    for k in par1.keys():
        par1[k].data.mul_(decay).add_(par2[k].data, alpha=1 - decay)


def train(
        train_dl,
        val_dl,
        model,
        model_ema,
        optimizer,
        criterion,
        num_steps,
        batch_size,
        device,
        initial_step=1,
        use_wandb=False,
        ema_decay=0.9995,
):
    pbar = range(num_steps)
    pbar = tqdm(pbar, initial=initial_step, dynamic_ncols=True)

    train_dl = generator(train_dl)
    val_dl = generator(val_dl)

    for step in pbar:
        step += initial_step

        images, labels = next(train_dl)
        images = images.to(device)
        labels = labels.to(device)

        prediction = model(images)
        loss = criterion(labels, prediction)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        weights_ema(model_ema, model, ema_decay)



In [104]:
# Fine-tuning hyperparameters
num_epochs = 30
batch_size = 128
learning_rate = 5e-6
weight_decay = 1e-8
label_smoothing = 0.1
ema_decay = 0.9995 # TODO: добавить функцию, которая делает ema_decay

# Dataset hyperparameters
width = 32
height = 32
channels = 3
num_classes = 10

# Augmentation hyperparameters
stochastic_depth_rate = [0.2, 0.4]
data_augmentation = "RandAugment"
alpha_mixup = 0.8
alpha_cutmix = 1.0
random_erase_prob = 0.25



train_transforms = 0
val_transforms = 0

train_ds = ImageDataset(labels_file=train_labels_path, root_dir=train_path, num_classes=10)
val_ds = ImageDataset(labels_file=val_labels_path, root_dir=val_path, num_classes=10)

train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=True)

In [105]:
len(train_dl)

391

In [263]:
model = timm.create_model("fastvit_s12", num_classes=num_classes, in_chans=channels)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)

#for epoch in range(num_epochs):


In [271]:
type(torch.nn.CrossEntropyLoss(label_smoothing=0.1))

torch.nn.modules.loss.CrossEntropyLoss

In [268]:
torchinfo.summary(model, input_size=[1, 3, 32, 32], depth=5)

Layer (type:depth-idx)                                                 Output Shape              Param #
FastVit                                                                [1, 10]                   --
├─Sequential: 1-1                                                      [1, 64, 8, 8]             --
│    └─MobileOneBlock: 2-1                                             [1, 64, 16, 16]           --
│    │    └─ConvNormAct: 3-1                                           [1, 64, 16, 16]           --
│    │    │    └─Conv2d: 4-1                                           [1, 64, 16, 16]           192
│    │    │    └─BatchNormAct2d: 4-2                                   [1, 64, 16, 16]           128
│    │    │    │    └─Identity: 5-1                                    [1, 64, 16, 16]           --
│    │    │    │    └─Identity: 5-2                                    [1, 64, 16, 16]           --
│    │    └─ModuleList: 3-2                                            --                    

In [358]:
timm.list_models("efficientnet_b0")

['efficientnet_b0']

In [108]:
timm.list_models("fast*")

['fastvit_ma36',
 'fastvit_s12',
 'fastvit_sa12',
 'fastvit_sa24',
 'fastvit_sa36',
 'fastvit_t8',
 'fastvit_t12']

In [109]:
timm.list_models("fast*", pretrained=True)

['fastvit_ma36.apple_dist_in1k',
 'fastvit_ma36.apple_in1k',
 'fastvit_s12.apple_dist_in1k',
 'fastvit_s12.apple_in1k',
 'fastvit_sa12.apple_dist_in1k',
 'fastvit_sa12.apple_in1k',
 'fastvit_sa24.apple_dist_in1k',
 'fastvit_sa24.apple_in1k',
 'fastvit_sa36.apple_dist_in1k',
 'fastvit_sa36.apple_in1k',
 'fastvit_t8.apple_dist_in1k',
 'fastvit_t8.apple_in1k',
 'fastvit_t12.apple_dist_in1k',
 'fastvit_t12.apple_in1k']

In [359]:
num_classes = 10
channels = 3
model = timm.create_model("efficientnet_b0", num_classes=num_classes, in_chans=channels)

In [360]:
model

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
      

In [361]:
model.default_cfg

{'url': 'https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b0_ra-3dd342df.pth',
 'hf_hub_id': 'timm/efficientnet_b0.ra_in1k',
 'architecture': 'efficientnet_b0',
 'tag': 'ra_in1k',
 'custom_load': False,
 'input_size': (3, 224, 224),
 'fixed_input_size': False,
 'interpolation': 'bicubic',
 'crop_pct': 0.875,
 'crop_mode': 'center',
 'mean': (0.485, 0.456, 0.406),
 'std': (0.229, 0.224, 0.225),
 'num_classes': 1000,
 'pool_size': (7, 7),
 'first_conv': 'conv_stem',
 'classifier': 'classifier'}

In [125]:
model(torch.randn((1, 3, 32, 32)))

tensor([[ 1.3654e-04, -6.3149e-05,  2.3090e-04, -1.4228e-04,  1.0160e-04,
         -4.9103e-04, -7.2727e-05,  1.5651e-04,  2.0577e-04,  3.3913e-04]],
       grad_fn=<MmBackward0>)

In [122]:
model.head

ClassifierHead(
  (global_pool): SelectAdaptivePool2d (pool_type=avg, flatten=Flatten(start_dim=1, end_dim=-1))
  (drop): Dropout(p=0.0, inplace=False)
  (fc): Sequential(
    (0): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Linear(in_features=1024, out_features=512, bias=False)
    (2): ReLU()
    (3): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Dropout(p=0.5, inplace=False)
    (5): Linear(in_features=512, out_features=10, bias=False)
  )
  (flatten): Identity()
)

In [113]:
# Custom final layer
num_in_features = model.get_classifier().in_features
model.head.fc = nn.Sequential(
    nn.BatchNorm1d(num_in_features),
    nn.Linear(in_features=num_in_features, out_features=512, bias=False),
    nn.ReLU(),
    nn.BatchNorm1d(512),
    nn.Dropout(0.5),
    nn.Linear(in_features=512, out_features=num_classes, bias=False)
)

In [114]:
model.eval()
model(torch.randn(1, 3, 256, 256)).shape

torch.Size([1, 10])

In [115]:
model.feature_info

[{'num_chs': 64, 'reduction': 4, 'module': 'stages.0'},
 {'num_chs': 128, 'reduction': 8, 'module': 'stages.1'},
 {'num_chs': 256, 'reduction': 16, 'module': 'stages.2'},
 {'num_chs': 512, 'reduction': 32, 'module': 'stages.3'}]

In [43]:
# Exporting to TorchScript
model = timm.create_model("fastvit_s12", scriptable=True)
model.eval()
scripted_model = torch.jit.script(model)

In [45]:
5e-6

5e-06